# Taller Flask

## 1. Despliegue de una API

Para este apartado vamos a realizar un primer despliegue de un servicio web que simplemente contenga un mensaje. Para ello hay que seguir los siguientes pasos:

1. Busca un lugar en tu ordenador y crea una carpeta de "demo_clase", donde alojarás todos los proyectos de estos tutoriales.
2. Crea un archivo de python llamado "app1.py"
3. Introduce el siguiente código. Primero se crea la configuración de la API Flask. Después viene declarado el único tipo de petición, que sería un GET a la URL del servicio raíz, el cual devuelve una página web, un HTML, con un par de frases. Finalmente, se queda corriendo la app mediante "app.run()".

In [ ]:
# Importamos la librería Flask para crear aplicaciones web
import flask

# Creamos una instancia de la aplicación Flask
# __name__ le indica a Flask el nombre del módulo actual
app = flask.Flask(__name__)

# Activamos el modo DEBUG para ver errores detallados durante el desarrollo
# IMPORTANTE: Nunca usar DEBUG=True en producción por seguridad
app.config["DEBUG"] = True

# Decorador @app.route: define la ruta URL y el método HTTP permitido
# '/' significa la ruta raíz (página principal)
# methods=['GET'] indica que solo acepta peticiones GET (solicitar datos)
@app.route('/', methods=['GET'])
def home():
    # Esta función se ejecuta cuando alguien visita la URL raíz
    # Retorna código HTML que se mostrará en el navegador
    return "<h1>Distant Reading Archive</h1><p>This site is a prototype API for distant reading of science fiction novels.</p>"

# Inicia el servidor web en el puerto 5000 por defecto
# La aplicación quedará esperando peticiones HTTP
app.run()

4. Para ejecutar esto, ve a un "Simbolo del sistema", "powershell" (o terminal si estás en MAC) y dentro de la carpeta "/demo_clase", ejecuta "python app1.py". Esto lanzará el servicio en local. Accede al mismo mediante la URL sugerida por el output de la sentencia (lo normal es que sea http://127.0.0.1:5000/).

Para que el servicio deje de correr: **CTRL + C**

Fíjate en el terminal cada vez que accedes a esa URL. El servicio está recibiendo peticiones, como si un usuario estuviera accediendo a una web.

### ¿Qué está haciendo Flask?

Está mapeando la URL "/", con la función "home()", por lo que cada vez que se acceda a la URL "/", es decir, sin ninguna ruta extra, llamará a la función "home()" y devolverá su output, que en este caso es un texto HTML, pero podría ser un JSON con otros datos.

El proceso de mapear URLs a funciones se denomina **routing**.

```python
@app.route('/', methods=['GET'])
```

Utiliza como methods, el GET, que es la acción HTTP para que el servidor devuelva datos al usuario. Se suele combinar mucho con POST, para recibir datos del usuario.

```python
app.config["DEBUG"] = True
```

Se utiliza para que salten los errores en la página y podamos ver bien qué es. Si no, pondría un "Bad Gateway".

## 2. API con datos

Como el propósito de este taller es montar una API con la que podamos acceder a una base de datos de libros, comenzaremos creando algunos datos sintéticos, así como un enrutado a los mismos:

1. Importa:

In [ ]:
# request: permite acceder a los datos enviados en las peticiones HTTP (parámetros, JSON, etc.)
# jsonify: convierte objetos Python (listas, diccionarios) en formato JSON para las respuestas
from flask import request, jsonify

2. Declara una lista con varios diccionarios. Éstos serán los datos que devolverá la API

In [ ]:
# Simulamos una base de datos con una lista de diccionarios
# En ciencia de datos, esto sería equivalente a un DataFrame con registros de libros
# Cada diccionario representa un libro con sus atributos (id, título, autor, etc.)
books = [
    {'id': 0,
     'title': 'A Fire Upon the Deep',
     'author': 'Vernor Vinge',
     'first_sentence': 'The coldsleep itself was dreamless.',
     'year_published': '1992'},
    {'id': 1,
     'title': 'The Ones Who Walk Away From Omelas',
     'author': 'Ursula K. Le Guin',
     'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.',
     'published': '1973'},
    {'id': 2,
     'title': 'Dhalgren',
     'author': 'Samuel R. Delany',
     'first_sentence': 'to wound the autumnal city.',
     'published': '1975'},
    {'id': 3,
     'title': 'The Chain',
     'author': 'Jaime G. Páramo',
     'first_sentence': 'There were tears on her eyes and fears trapped her mind but, inside, the courage of those who have nothing to lose and all to win, flown wild and free.',
     'published': '2025'}
]

3. Añade la ruta y función para acceder a estos datos

In [ ]:
# Endpoint para obtener TODOS los libros de nuestra "base de datos"
# Ruta: /api/v1/resources/books/all
@app.route('/api/v1/resources/books/all', methods=['GET'])
def api_all():
    # jsonify() convierte la lista de diccionarios Python a formato JSON
    # JSON es el estándar para transferir datos en APIs REST
    return jsonify(books)

4. Corre la aplicación y accede a http://127.0.0.1:5000/api/v1/resources/books/all

Verás que la API devuelve un JSON, que es el formato de datos más común para comunicaciones web. Este json lo creamos a partir de la lista de diccionarios, gracias a la función "jsonify()", de flask.

De momento nuestro programa tiene un punto de acceso y devuelve todos los libros de nuestra "base de datos".

## 3. API para búsqueda de datos

Hasta el momento hemos creado un endpoint y una ruta de acceso a todos los datos. Para este apartado implementaremos en la API una búsqueda de objetos por ID:

1. Añade la siguiente función para buscar por ID. Comprueba en los argumentos de la petición si existe ID. De ser así, buscamos en la base de datos con ese ID, y en caso contrario, devolvemos un mensaje de error.

In [ ]:
# Endpoint para buscar un libro por su ID usando query parameters
# Ejemplo de uso: /api/v1/resources/book?id=2
@app.route('/api/v1/resources/book', methods=['GET'])
def api_id():
    # Verificamos si el parámetro 'id' fue enviado en la URL
    # request.args contiene los query parameters (lo que va después del ?)
    if 'id' in request.args:
        # Convertimos el id de string a entero
        id = int(request.args['id'])
    else:
        # Si no hay id, retornamos un mensaje de error
        return "Error: No id field provided. Please specify an id."
    
    # Lista para almacenar los resultados encontrados
    results = []
    
    # Iteramos sobre todos los libros (similar a filtrar un DataFrame en pandas)
    for book in books:
        # Si encontramos un libro con el id buscado
        if book['id'] == id:
            results.append(book)
    
    # Retornamos los resultados en formato JSON
    return jsonify(results)

Accede a las siguientes URLs para comprobar su funcionamiento:
- 127.0.0.1:5000/api/v1/resources/books?id=0
- 127.0.0.1:5000/api/v1/resources/books?id=1
- 127.0.0.1:5000/api/v1/resources/books?id=2
- 127.0.0.1:5000/api/v1/resources/books?id=3

Todo lo que va después del "?", se denominan **query parameters**, empleados para filtrar un tipo de datos concreto.

En este punto tenemos creado un nuevo enrutado: "/api/v1/resources/books", que llamará a la función "api_id()", cada vez que se acceda a esa ruta.

Otra manera de acceder a los datos es con el argumento en la propia URL. Como en el siguiente ejemplo donde vamos a buscar por título:

In [ ]:
# Búsqueda con parámetros en la URL (path parameters en lugar de query parameters)
# Ejemplo: /api/v1/resources/book/Dhalgren
# <string:title> captura el valor de la URL y lo pasa como argumento a la función
@app.route('/api/v1/resources/book/<string:title>', methods=['GET'])
def get_by_title(title):
    # Buscamos el libro que coincida con el título exacto
    for book in books:
        if book['title'] == title:
            # Si lo encontramos, lo retornamos inmediatamente
            return jsonify(book)
    # Si no se encuentra ningún libro, retornamos un mensaje
    return jsonify({'message': "Book not found"})

O también con los argumentos en el propio cuerpo de la petición HTTP. Fíjate que se ha cambiado la version del enrutado, para no confundir con los anteriores.

In [ ]:
# Búsqueda con datos enviados en el CUERPO de la petición (body)
# v2 indica una versión diferente del API
@app.route('/api/v2/resources/book', methods=['GET'])
def get_by_id():
    # request.get_json() extrae los datos JSON del cuerpo de la petición
    # Útil cuando se envían datos más complejos desde el cliente
    id = int(request.get_json()['id'])
    
    # Mismo proceso de búsqueda
    for book in books:
        if book['id'] == id:
            return jsonify(book)
    return jsonify({'message': "Book not found"})

Si quisiésemos subir a la BD algún libro, realizaremos un POST:

In [ ]:
# Método POST para AGREGAR un nuevo libro a la base de datos
# POST se usa cuando queremos crear/insertar nuevos datos (diferente a GET que solo consulta)
@app.route('/api/v1/resources/book', methods=['POST'])
def post_book():
    # Obtenemos el JSON enviado por el cliente con los datos del nuevo libro
    data = request.get_json()
    
    # Agregamos el nuevo libro a nuestra lista (en producción sería INSERT en BD)
    books.append(data)
    
    # Retornamos el libro agregado como confirmación
    return data

## 4. API con BD SQL

Finalmente crearemos una API que sea capaz de manejar errores de consulta a la base de datos, descargarse todos los libros y filtrarlos por fecha de publicación. La BD se puede descargar desde este enlace. Utilizaremos el siguiente código:

In [ ]:
# Importamos Flask y request para manejar peticiones HTTP
from flask import Flask, request

# sqlite3 es la librería estándar de Python para bases de datos SQLite
# SQLite es una BD ligera ideal para proyectos pequeños y medianos
import sqlite3

# Inicializamos la aplicación Flask
app = Flask(__name__)
app.config["DEBUG"] = True

In [ ]:
# Endpoint para obtener TODOS los libros desde una base de datos SQLite real
@app.route('/api/v1/resources/books/all', methods=['GET'])
def get_all():
    # Establecemos conexión con la base de datos (archivo books.db)
    connection = sqlite3.connect('books.db')
    
    # Creamos un cursor: objeto que nos permite ejecutar consultas SQL
    cursor = connection.cursor()
    
    # Query SQL: SELECT * significa "seleccionar todas las columnas"
    # FROM books: de la tabla llamada "books"
    select_books = "SELECT * FROM books"
    
    # Ejecutamos la consulta y obtenemos TODOS los resultados con fetchall()
    # fetchall() retorna una lista de tuplas, cada tupla es un registro
    result = cursor.execute(select_books).fetchall()
    
    # IMPORTANTE: Siempre cerrar la conexión para liberar recursos
    connection.close()
    
    # Retornamos los resultados en formato JSON
    return {'books': result}

In [ ]:
# Búsqueda de libros por autor usando path parameter
# Ejemplo: /api/v1/resources/book/Vernor%20Vinge
@app.route('/api/v1/resources/book/<string:author>', methods=['GET'])
def get_by_author(author):
    # Conectamos a la base de datos
    connection = sqlite3.connect('books.db')
    cursor = connection.cursor()
    
    # Query SQL con filtro WHERE
    # El "?" es un placeholder (marcador de posición) para prevenir SQL injection
    # NUNCA concatenar variables directamente en SQL por seguridad
    select_books = "SELECT * FROM books WHERE author=?"
    
    # Pasamos el valor del autor como tupla (author,) al segundo argumento
    # SQLite reemplazará el ? con este valor de forma segura
    result = cursor.execute(select_books, (author,)).fetchall()
    
    connection.close()
    
    return {'books': result}

In [ ]:
# Endpoint avanzado: filtrado dinámico por múltiples campos
# Permite filtrar por id, published, author o combinación de ellos
@app.route('/api/v1/resources/book/filter', methods=['GET'])
def filter_table():
    # Obtenemos los parámetros del cuerpo de la petición JSON
    query_parameters = request.get_json()
    
    # Extraemos cada parámetro posible (pueden venir o no)
    # .get() retorna None si la clave no existe (más seguro que ['key'])
    id = query_parameters.get('id')
    published = query_parameters.get('published')
    author = query_parameters.get('author')
    
    # Conectamos a la BD
    connection = sqlite3.connect('books.db')
    cursor = connection.cursor()
    
    # Construimos la query SQL dinámicamente según los filtros recibidos
    query = "SELECT * FROM books WHERE"
    to_filter = []  # Lista para almacenar los valores de los filtros
    
    # Si el cliente envió 'id', lo agregamos a la query
    if id:
        query += ' id=? AND'  # Agregamos condición con placeholder
        to_filter.append(id)  # Agregamos el valor a la lista
    
    # Si el cliente envió 'published', lo agregamos
    if published:
        query += ' published=? AND'
        to_filter.append(published)
    
    # Si el cliente envió 'author', lo agregamos
    if author:
        query += ' author=? AND'
        to_filter.append(author)
    
    # Si no se envió ningún filtro, retornamos error 404
    if not (id or published or author):
        return "page not found 404"
    
    # Eliminamos el último " AND" (4 caracteres) y agregamos punto y coma
    # query[:-4] toma todo menos los últimos 4 caracteres
    query = query[:-4] + ';'
    
    # Ejecutamos la query con los valores de filtro
    # to_filter se expande en los placeholders "?" en orden
    result = cursor.execute(query, to_filter).fetchall()
    
    connection.close()
    
    return {'books': result}

In [ ]:
# Iniciamos el servidor Flask
# Por defecto corre en http://127.0.0.1:5000/
# CTRL+C para detener el servidor
app.run()

Prueba las siguientes URLs:
- http://127.0.0.1:5000/api/v1/resources/books/all
- http://127.0.0.1:5000/api/v1/resources/book/Connie%20Willis

La BD tiene 67 entradas, con libros ganadores del premio Hugo entre 1953 y 2014. Incluye nombres de las novelas, autor, id, año de publicación y primera frase.

La función "dict_factory()" permite devolver los objetos de la BD como diccionarios, y no como listas.

## 5. Material extra

Si quieres seguir formándote con Flask, estos recursos te pueden ayudar:

**Tutorial completo y nivel avanzado**
https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world

**Curso con videos de Oreilly**
https://learning.oreilly.com/videos/rest-apis-with/9781788621526/

**Flask Web Development - Oreilly**
https://learning.oreilly.com/library/view/flask-web-development/9781491991725/